In [62]:
import pandas as pd

#lectura de datasets
samsung_df = pd.read_csv('SSNG.csv', delimiter=',')
apple_df = pd.read_csv('AAPL.csv', delimiter=',')
won_to_usd_00_17_df = pd.read_csv('KRW_TO_USD_2000-2017.csv', delimiter=',')
won_to_usd_04_22_df = pd.read_csv('KRW_TO_USD_2004-2022.csv', delimiter=',')
won_to_usd_04_22_df = won_to_usd_04_22_df[['Date', 'KRW=X']]

#eliminar filas con valores nulos
samsung_df = samsung_df.dropna()
apple_df = apple_df.dropna()
won_to_usd_00_17_df = won_to_usd_00_17_df.dropna()
won_to_usd_04_22_df = won_to_usd_04_22_df.dropna()

won_to_usd_00_17_df = won_to_usd_00_17_df[won_to_usd_00_17_df['DEXKOUS'] != '.']
won_to_usd_04_22_df = won_to_usd_04_22_df[won_to_usd_04_22_df['KRW=X'] != '.']


#formateando las columnas de fechas
samsung_df['Date'] = pd.to_datetime(samsung_df['Date'])
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
won_to_usd_00_17_df['DATE'] = pd.to_datetime(won_to_usd_00_17_df['DATE'])
won_to_usd_04_22_df['Date'] = pd.to_datetime(won_to_usd_04_22_df['Date'])

samsung_df['Date'] = samsung_df['Date'].dt.strftime('%d-%m-%Y')
apple_df['Date'] = apple_df['Date'].dt.strftime('%d-%m-%Y')
won_to_usd_00_17_df['DATE'] = won_to_usd_00_17_df['DATE'].dt.strftime('%d-%m-%Y')
won_to_usd_04_22_df['Date'] = won_to_usd_04_22_df['Date'].dt.strftime('%d-%m-%Y')


#renombrado de columnas
samsung_df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'}, inplace=True)
apple_df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'}, inplace=True)
won_to_usd_00_17_df.rename(columns={'DATE': 'date', 'DEXKOUS': 'dollar'}, inplace=True)
won_to_usd_04_22_df.rename(columns={'Date': 'date', 'KRW=X': 'dollar'}, inplace=True)

# ----- definición de índice --------
# samsung_df.set_index('date', inplace=True)
# apple_df.set_index('date', inplace=True)
# won_to_usd_00_17_df.set_index('date', inplace=True)
# won_to_usd_04_22_df.set_index('date', inplace=True)

#mostrar datasets
#print(samsung_df.head())
#print('\n\n\n-----------------------------------------\n\n')
#print(apple_df.head())

#print(won_to_usd_04_22_df.sort_values('dollar'))
#print('\n\n\n-----------------------------------------\n\n')
#print(won_to_usd_00_17_df.sort_values('dollar'))


#combinar datasets de valores de conversion entre dolar y won coreano
combined_dollar_values = pd.concat([won_to_usd_00_17_df, won_to_usd_04_22_df])
combined_dollar_values = combined_dollar_values.drop_duplicates(subset='date')


#agregar columna de conversion de won a dolar
samsung_df['usd_to_won'] = samsung_df['date'].map(combined_dollar_values.set_index('date')['dollar'])


#filtrar filas con volumen 0 y eliminar filas con valores nulos
samsung_df = samsung_df[samsung_df['volume'] != 0]
samsung_df = samsung_df.dropna()


#convertir valores de won a dolar en la tabla de samsung
samsung_df['open'] = samsung_df.apply(lambda x: float(x['open']) / float(x['usd_to_won']), axis=1)
samsung_df['high'] = samsung_df.apply(lambda x: float(x['high']) / float(x['usd_to_won']), axis=1)
samsung_df['low'] = samsung_df.apply(lambda x: float(x['low']) / float(x['usd_to_won']), axis=1)
samsung_df['close'] = samsung_df.apply(lambda x: float(x['close']) / float(x['usd_to_won']), axis=1)
samsung_df['adj_close'] = samsung_df.apply(lambda x: float(x['adj_close']) / float(x['usd_to_won']), axis=1)

samsung_df['year'] = pd.DatetimeIndex(samsung_df['date']).year
print(samsung_df.sort_values('year', ascending=True))


            date       open       high        low      close  adj_close  \
20    01-02-2000   4.968944   5.039929   4.667258   4.720497   3.593865   
39    28-02-2000   4.405286   4.449339   4.229075   4.229075   3.219730   
75    18-04-2000   5.585586   5.621622   5.279279   5.441441   4.142743   
96    17-05-2000   6.274650   6.579419   6.202940   6.202940   4.722496   
97    18-05-2000   6.024312   6.256704   5.970683   6.077941   4.627330   
...          ...        ...        ...        ...        ...        ...   
5615  16-05-2022  52.521584  52.756405  51.738848  51.895395  51.895395   
5548  04-02-2022  61.899644  62.149576  61.149850  61.649713  61.332684   
5599  20-04-2022  54.049693  54.372378  53.646338  54.372378  54.372378   
5605  28-04-2022  51.647345  51.726316  50.936602  51.173516  51.173516   
5571  11-03-2022  57.382386  57.545173  56.731239  56.975419  56.682428   

        volume usd_to_won  year  
20    71470000    1127.00  2000  
39    30140000    1135.00  2000